In [ ]:

!apt-get install libnvidia-gl-$(grep -oP 'NVIDIA UNIX x86_64 Kernel Module\s+\K[\d.]+(?=\s+)' /proc/driver/nvidia/version | grep -oE '^[0-9]+')


In [ ]:
%%file requirements.txt
glcontext
moderngl
numpy
opencv-python
websockets


In [ ]:

!pip install -r requirements.txt
!pip install pyngrok ping3


In [ ]:

!python -m moderngl


In [ ]:

!mkdir -p app
!mkdir -p assets/glsl
!mkdir -p colab


In [ ]:
%%file assets/glsl/compute_shader.glsl
#version 430

#define POW2(X) ((X) * (X))
#define POW5(X) ((X) * (X) * (X) * (X) * (X))
#define DEPTH_MAX (16)
#define SAMPLE_MAX ($sample_max)
#define DELTA (0.01)
#define PI (3.14159265359)

#define BACKGROUND (0)
#define DIFFUSE (1)
#define MIRROR (2)
#define GLASS (3)

layout(local_size_x = $local_size_x, local_size_y = $local_size_y) in;

uniform int current_sample;
uniform float theta;
uniform float phi;
uniform float move_x;
uniform float move_y;

layout(binding = 1, rgba32f) uniform image2D input_image;
layout(binding = 2, rgba32f) uniform image2D output_image;
layout(binding = 3, rgba32ui) uniform uimage2D seed_image;
layout(binding = 4) uniform sampler2D background_image;

ivec3 group_num = ivec3(
	gl_NumWorkGroups.x * gl_WorkGroupSize.x,
	gl_NumWorkGroups.y * gl_WorkGroupSize.y,
	gl_NumWorkGroups.z * gl_WorkGroupSize.z
);
ivec3 group_idx = ivec3(
	gl_GlobalInvocationID.x,
	gl_GlobalInvocationID.y,
	gl_GlobalInvocationID.z
);

struct Ray {
    vec3 origin;    // 光線の始点
    vec3 direction; // 光線の方向ベクトル
    vec3 scatter;   // 散乱成分
    uint depth;     // 反射した回数
};

struct Hit {
    float t;        // 光線の始点から衝突位置までの距離
    vec3 position;  // 衝突位置
    vec3 normal;    // 衝突位置における法線ベクトル
    vec3 scatter;   // 散乱成分
    vec3 emission;  // 放出成分
    uint material;  // 材質
};

struct Sphere {
    vec3 center;    // 球の中心
    float radius;   // 球の半径
    vec3 scatter;   // 散乱成分
    vec3 emission;  // 放出成分
    uint material;  // 材質
};

uvec4 xors;

// Xorshift による疑似乱数生成
float rand() {
    uint t = (xors[0] ^ (xors[0] << 11));
    xors[0] = xors[1];
    xors[1] = xors[2];
    xors[2] = xors[3];
    xors[3] = (xors[3] ^ (xors[3] >> 19)) - (t ^ (t >> 18));
    return xors[3] / 4294967295.0f;
}

// 球と光線の交点
bool hitSphere(const in Sphere sphere, const in Ray ray, inout Hit hit) {
    vec3 oc = ray.origin - sphere.center;
    float a = dot(ray.direction, ray.direction);
    float b = dot(oc, ray.direction);
    float c = dot(oc, oc) - POW2(sphere.radius);
    float d = POW2(b) - a * c;

    float t;
    if (d > 0)
    {
        t = (-b - sqrt(d)) / a;
        if (0 < t && t < hit.t)
        {
            hit.t = t;
            hit.position = ray.origin + t * ray.direction;
            hit.normal = normalize(hit.position - sphere.center);
            hit.scatter = sphere.scatter;
            hit.emission = sphere.emission;
            hit.material = sphere.material;
            return true;
        }
        t = (-b + sqrt(d)) / a;
        if (0 < t && t < hit.t)
        {
            hit.t = t;
            hit.position = ray.origin + t * ray.direction;
            hit.normal = normalize(hit.position - sphere.center);
            hit.scatter = sphere.scatter;
            hit.emission = sphere.emission;
            hit.material = sphere.material;
            return true;
        }
    }

    return false;
}

// 鏡面
void mirror(inout Ray ray, const in Hit hit)
{
	if (dot(-ray.direction, hit.normal) < 0)
	{
        ray.depth = DEPTH_MAX;
		return;
	}
    ray.depth++;
	ray.origin = hit.position + hit.normal * DELTA;
    // 課題1：鏡面の作成
	// ray.direction =
	ray.scatter *= hit.scatter;
}

float fresnel(const in float n, const in float u)
{
	const float f0 = POW2((n - 1) / (n + 1));
	return f0 + (1 - f0) * POW5(1 - u);
}

// ガラス面
void glass(inout Ray ray, const in Hit hit)
{
    ray.depth++;
	float n = 1.5;
	vec3 N;
	float t = dot(-ray.direction, hit.normal);
	if (t > 0.0f)
	{
		n = 1.0f / n;
		N = hit.normal;
		t = t;
	}
	else
	{
		n = n / 1.0f;
		N = -hit.normal;
		t = -t;
	}
	if (rand() < fresnel(n, t) || n * length(cross(N, -ray.direction)) > 1)
	{
		mirror(ray, hit);
	}
	else
	{
		ray.origin = hit.position - N * DELTA;
        // 課題2：ガラス面の作成
		// ray.direction =
		ray.scatter *= hit.scatter;
	}
}

// Image Based Lighting
void background(inout Ray ray, inout Hit hit) {
    ray.depth = DEPTH_MAX;
    hit.emission = texture(
        background_image,
        vec2(
            -atan(ray.direction.x, ray.direction.z) / (2 * PI),
             acos(ray.direction.y) / PI
        )
    ).rgb;
}

// 完全拡散反射面
void diffuse(inout Ray ray, const in Hit hit) {
    if (dot(-ray.direction, hit.normal) < 0) {
        ray.depth = DEPTH_MAX;
        ray.scatter = vec3(0.0f);
        return;
    }
    ray.depth++;
    ray.direction.y = sqrt(rand());
    float d = sqrt(1 - POW2(ray.direction.y));
    float v = rand() * 2.0f * PI;
    vec3 ex = vec3(1.0f, 0.0f, 0.0f);
    vec3 ey = vec3(0.0f, 1.0f, 0.0f);
    vec3 ez = vec3(0.0f, 0.0f, 1.0f);
    float dx = abs(dot(hit.normal, ex));
    float dy = abs(dot(hit.normal, ey));
    float dz = abs(dot(hit.normal, ez));
    vec3 vy = (dy < dx) ? (dz < dy) ? ez : ey : (dz < dx) ? ez : ex;
    vec3 vx = normalize(cross(vy, hit.normal));
    vec3 vz = normalize(cross(vx, hit.normal));

    ray.direction = normalize(vx * d * cos(v) + hit.normal * ray.direction.y + vz * d * sin(v));
    ray.origin = hit.position + hit.normal * DELTA;
    ray.scatter *= hit.scatter;
}

// Tone Mapping
vec4 toneMap(const in vec4 color, const in float white) {
    return clamp(color * (1 + color / white) / (1 + color), 0, 1);
}

// Gamma Correction
vec4 gammaCorrect(const in vec4 color, const in float gamma) {
    float c = 1.0f / gamma;
    return vec4(pow(color.r, c), pow(color.g, c), pow(color.b, c), 0.0f);
}

void main() {
    xors = imageLoad(seed_image, group_idx.xy);

    vec4 color_present = (current_sample == 1) ? vec4(0.0f) : imageLoad(input_image, group_idx.xy);

    const vec3 eye = vec3(0.0f, 0.0f, 18.0f);

    const uint n_sphere = 2;
    const Sphere spheres[n_sphere] = {
        {
            vec3(0.0f),
            4.0f,
            vec3(0.75f),
            vec3(0),
            DIFFUSE,
        },
        {
            vec3(0.0f, -10000.05f, 0.0f),
            9996.0f,
            vec3(0.75f),
            vec3(0),
            DIFFUSE,
        },
    };

    const mat3 M1 = mat3(
        cos(theta), 0, sin(theta),
        0, 1, 0,
        -sin(theta), 0, cos(theta)
    );

	const mat3 M2 = mat3(
		1, 0, 0,
		0, cos(phi), -sin(phi),
		0, sin(phi), cos(phi)
	);

    for (int i = 0; i < SAMPLE_MAX; i++) {
        vec4 color_next = vec4(0.0f);

        const vec3 position_screen = {
            float(group_idx.x + rand()) / group_num.x * 16.0f - 8.0f,
            float(group_idx.y + rand()) / group_num.y *  9.0f - 4.5f,
            eye.z - 9.0f,
        };

        Ray ray = {
            M1 * M2 * (eye + vec3(move_x, move_y, 0)),
            M1 * M2 * (normalize(position_screen - eye)),
            vec3(1.0f),
            0,
        };

        Hit hit = {
            1000.0f,
            vec3(0.0f),
            vec3(0.0f),
            vec3(0.0f),
            vec3(0.0f),
            BACKGROUND,
        };

        while (ray.depth < DEPTH_MAX) {
            for (int i = 0; i < n_sphere; i++) {
                hitSphere(spheres[i], ray, hit);
            }

            switch (hit.material) {
                case BACKGROUND: background(ray, hit); break;
                case DIFFUSE: diffuse(ray, hit); break;
                case MIRROR: mirror(ray, hit); break;
                case GLASS: glass(ray, hit); break;
            }

            color_next.rgb += hit.emission * ray.scatter;

            hit.t = 10000.0f;
            hit.material = BACKGROUND;
        }

        // 平均値の逐次計算
        color_present += (color_next - color_present) / (current_sample + i);
    }

    imageStore(input_image, group_idx.xy, color_present);
    imageStore(output_image, group_idx.xy, gammaCorrect(toneMap(color_present, 1000.0f), 2.2));
    imageStore(seed_image, group_idx.xy, xors);
}


In [ ]:
%%file app/server.py
import asyncio
import json

from websockets.server import serve

from render import Context


class WebSocket:
    def __init__(self, context):
        self.context = context

    async def task(self, websocket):
        # キャンセルされるまでサンプリングとレンダリング結果画像の送信を繰り返す
        self.context.create_shader()
        i = 0
        while True:
            print(["-", "/", "|", "\\"][i % 4], "\r", end="")
            try:
                self.context.current_sample = i * self.context.sample_per_frame + 1
                i += 1
                next_frame = i * self.context.sample_per_frame

                if self.context.max_spp:
                    if next_frame > int(self.context.max_spp):
                        self.context.create_shader(
                            int(self.context.max_spp) % self.context.sample_per_frame
                        )
                        next_frame = int(self.context.max_spp)

                self.context.render()

                await asyncio.gather(
                    # レンダリング結果画像を送信する（識別子：0000）
                    websocket.send(b"0000" + self.context.get_binary().getvalue()),
                    # 現在の1画素あたりのサンプル数を送信する（識別子：0001）
                    websocket.send(b"0001" + bytes(next_frame)),
                )

                if self.context.max_spp:
                    if next_frame >= int(self.context.max_spp):
                        break
            except RuntimeError as e:
                print("Runtime Error:", e)
                break
            except ValueError as e:
                print("ValueError:", e)
                break

    async def echo(self, websocket):
        current_task = None
        print("init")
        print("current_task: ", current_task)

        # クライアントからの接続要求を待ち受ける
        while True:
            message = json.loads(await websocket.recv())
            if "theta" in message:
                self.context.theta = message["theta"]
            if "phi" in message:
                self.context.phi = message["phi"]
            if "moveX" in message:
                self.context.move_x = message["moveX"]
            if "moveY" in message:
                self.context.move_y = message["moveY"]
            if "maxSpp" in message:
                self.context.max_spp = message["maxSpp"]

            for task in asyncio.all_tasks():
                if task.get_coro().__name__ == "task" and not task.done():
                    task.cancel()

            # レンダリングタスクを実行する
            current_task = asyncio.create_task(self.task(websocket))

            print("task assigned")
            print("current_task: ", current_task)

    async def main(self, host, port):
        async with serve(self.echo, host, port):
            print("Listening at: ", f"ws://{host}:{port}")
            await asyncio.Future()  # run forever


if __name__ == "__main__":
    ctx = Context(
        width=960, height=540, local_size_x=8, local_size_y=4, sample_per_frame=64
    )
    ctx.bind_data(env_map_path="assets/hdr/museum_of_ethnography_1k.hdr")
    ws = WebSocket(ctx)
    asyncio.run(ws.main("127.0.0.1", 8030))


In [ ]:
%%file app/render.py
import io
import platform
from string import Template

import cv2
import numpy as np
import moderngl


class Context:
    def __init__(
        self, width=960, height=540, local_size_x=8, local_size_y=4, sample_per_frame=1
    ):
        kwargs = {
            "standalone": True,
        }
        if platform.system() == "Linux":
            kwargs["backend"] = "egl"
        self.context = moderngl.create_context(**kwargs)

        self.width = width
        self.height = height
        self.local_size_x = local_size_x
        self.local_size_y = local_size_y
        self.sample_per_frame = sample_per_frame

        self.current_sample = 1
        self.theta = 0
        self.phi = 0
        self.move_x = 0
        self.move_y = 0
        self.max_spp = 0

        self.output_image = None

        self.compute_shader = None

    def bind_data(self, env_map_path):
        data = np.zeros((self.height, self.width, 4)).astype("float32").tobytes()

        # サンプリングを再開するために用いる raw 画像
        input_image = self.context.texture(
            (self.width, self.height), 4, data, dtype="f4"
        )
        input_image.bind_to_image(1)

        # 送信用画像（トーンマップおよびガンマ変換適用済み）
        self.output_image = self.context.texture(
            (self.width, self.height), 4, data, dtype="f4"
        )
        self.output_image.bind_to_image(2)

        # 乱数のシード画像（各画素で別々のシード値を使用）
        seed_image = self.context.texture(
            (self.width, self.height), 4, data, dtype="u4"
        )
        seed_image.write(
            data=np.random.default_rng()
            .integers(
                low=0, high=2**32, size=(self.width, self.height, 4), dtype=np.uint32
            )
            .tobytes()
        )
        seed_image.bind_to_image(3)

        # 環境マップ画像
        env_map = cv2.imread(env_map_path, cv2.IMREAD_UNCHANGED)
        env_map = cv2.cvtColor(env_map, cv2.COLOR_BGRA2RGBA)
        env_map = env_map.reshape((env_map.shape[1], env_map.shape[0], 4))
        background_img = self.context.texture(
            (env_map.shape[0], env_map.shape[1]), 4, env_map, dtype="f4"
        )
        background_img.write(data=env_map.astype("float32").tobytes())
        self.context.sampler(texture=background_img).use(4)

    def create_shader(self, sample_max=None):
        self.compute_shader = self.context.compute_shader(
            Template(
                open("assets/glsl/compute_shader.glsl", encoding="utf-8").read()
            ).substitute(
                width=self.width,
                height=self.height,
                local_size_x=self.local_size_x,
                local_size_y=self.local_size_y,
                sample_max=sample_max or self.sample_per_frame,
            )
        )

    def render(self):
        if self.compute_shader is None:
            raise RuntimeError("compute_shader has not been assigned")

        self.compute_shader["current_sample"].value = self.current_sample
        self.compute_shader["theta"].value = self.theta
        self.compute_shader["phi"].value = self.phi
        self.compute_shader["move_x"].value = self.move_x
        self.compute_shader["move_y"].value = self.move_y

        self.compute_shader.run(
            group_x=self.width // self.local_size_x + 1,
            group_y=self.height // self.local_size_y + 1,
        )

    def get_binary(self):
        if self.output_image is None:
            raise RuntimeError("output_image has not been assigned")

        buffer = np.frombuffer(self.output_image.read(), dtype="float32").reshape(
            self.height, self.width, 4
        )
        buffer = np.flipud(buffer)
        buffer = cv2.cvtColor(buffer, cv2.COLOR_BGRA2RGBA)
        buffer = (buffer * 255).astype(np.uint8)
        is_success, binary = cv2.imencode(".jpg", buffer)
        return io.BytesIO(binary)


In [ ]:

!(mkdir -p assets/hdr && cd assets/hdr && curl -O https://dl.polyhaven.org/file/ph-assets/HDRIs/hdr/1k/museum_of_ethnography_1k.hdr)


In [ ]:

import json
from pprint import pprint
from urllib import request

ipinfo = json.loads(request.urlopen("https://ipinfo.io").read())
pprint(ipinfo)


In [ ]:
%%file colab/tunnel.py
from ping3 import ping
from pyngrok import conf, ngrok


class Tunnel:
    # Ngrok がサポートするリージョンのリスト
    # cf.) https://ngrok.com/docs/ngrok-agent/config/#region
    region_list = ["us", "eu", "ap", "au", "sa", "jp", "in"]

    def __init__(self):
        self.region_priority_list = Tunnel.region_list
        self.auth_token = None

    def install_auth_token(self, auth_token=None):
        if auth_token is None:
            self.auth_token = input()
        else:
            self.auth_token = auth_token

    def calc_region_priority(self):
        key_list = []
        for region in Tunnel.region_list:
            public_url = self.get_public_url(region=region)
            key = float("inf")
            key = ping(public_url.replace("tcp://", "").split(":")[0])
            ngrok.disconnect(public_url)
            ngrok.kill()
            key_list.append(key)

        self.region_priority_list, _ = zip(
            *sorted(zip(Tunnel.region_list, key_list), key=lambda x: x[1])
        )

        print(self.region_priority_list, _)

    def get_public_url(self, port=80, region=None):
        if region == None:
            region = self.region_priority_list[0]
        pyngrok_config = conf.PyngrokConfig(region=region, auth_token=self.auth_token)
        return ngrok.connect(
            port, proto="tcp", pyngrok_config=pyngrok_config
        ).public_url


In [ ]:

from colab.tunnel import Tunnel

tunnel = Tunnel()

# Ngrok authtoken を指定
# authtoken の値をダッシュボード (https://dashboard.ngrok.com/get-started/your-authtoken) から取得し，フォームに入力する
# 毎回の入力を省略する場合は，authtoken の値を引数として渡す
tunnel.install_auth_token()

# Google Colaboratory のサーバと最も低遅延で通信できるトンネルを見つける
tunnel.calc_region_priority()

# グローバルアクセス可能なURLを取得する
public_url = tunnel.get_public_url(port=8030)
print(public_url.replace("tcp://", "ws://"))


In [ ]:

!python app/server.py
